## Querying data with DuckDB

### Import packages

In [1]:
from duckdb_utils.query_parquet import setup_duckdb_connection, load_parquet_from_bucket
import os
from dotenv import load_dotenv
load_dotenv()

True

### Load your credentials

In [2]:
ACCESS = os.getenv("GCS_ACCESS_KEY")
SECRET = os.getenv("GCS_SECRET")
BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")
BUCKET_PATH_BRONZE = os.getenv("GCS_BUCKET_PATH_BRONZE")
BUCKET_PATH_SILVER = os.getenv("GCS_BUCKET_PATH_SILVER")
BUCKET_PATH_GOLD = os.getenv("GCS_BUCKET_PATH_GOLD")

### Connect

In [3]:
duckdb_conn = setup_duckdb_connection(ACCESS, SECRET)

### Load parquet files

In [4]:
table_name = 'gold_table'
bucket_path = f"cornershop-raw/gold/year=2023/month=*/day=*/*.parquet"
load_parquet_from_bucket(duckdb_conn, table_name=table_name, bucket_path=bucket_path)

### Read the data with Pandas

In [5]:
df_gold = duckdb_conn.sql(f"SELECT * FROM {table_name}").df()

In [6]:
df_gold.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244028 entries, 0 to 244027
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          244028 non-null  datetime64[ns]
 1   aisle_name    244028 non-null  object        
 2   product_name  244028 non-null  object        
 3   brand         244028 non-null  object        
 4   package       244028 non-null  object        
 5   search_term   244028 non-null  object        
 6   price         244028 non-null  float64       
 7   store_name    244028 non-null  object        
 8   store_city    244028 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 16.8+ MB


In [7]:
df_gold.sample(10)

,date,aisle_name,product_name,brand,package,search_term,price,store_name,store_city
226505,2023-08-09,Lanches e acompanhamentos congelados,Steak de frango empanado congelado,Perdigão,100g,empanado,2.59,Carrefour Hiper,FLN
178458,2023-08-07,Balas de gelatina,Bala de gelatina sortidas sabor de frutas Aviões,Haribo,Pacote 90g,haribo,3.99,Atacadão,FLN
190634,2023-08-07,Shampoo e condicionador,Shampoo anticaspa anticoceira,Head & Shoulders,400ml,cabelo,27.89,Farmácia Preço Popular,FLN
94259,2023-08-02,Sucos e néctares,Néctar de abacaxi,Del Valle,1l,abacaxi,7.59,BIG by Carrefour,FLN
32473,2023-07-28,Alimento seco para gatos,Ração seca sabor frango para gatos adultos Fri...,Purina,1kg,frango,31.99,BIG by Carrefour,FLN
154413,2023-08-05,Comida e fórmula bebês,"Cereal infantil banana, maçã e quinoa",Mucilon,180g,cevada,9.12,Pague Menos,FLN
22937,2023-07-28,Chocolate,Chocolate ao leite com amendoim Shot,Lacta,Barra 80g,amendoim,6.45,Hippo,FLN
145233,2023-08-05,Vitaminas e suplementos,Colágeno hidrolisado Q10+Vit.C,Fixa Derme,15x3 gomas,cerveja,80.99,Droga Raia,FLN
31830,2023-07-28,Cerveja,Cerveja de trigo weizenbier,Eisenbahn,Garrafa 355ml,trigo,6.69,Carrefour Hiper,FLN
109718,2023-08-03,Carne de porco,Bisteca suína congelada,Sadia,Embalagem de 500g aprox.,porco,10.30,Carrefour Hiper,FLN


### Finding the Most Observed Product Name Across Different Dates

In [73]:
# Group by 'product_name' and count distinct dates
product_date_counts = df_gold.groupby('product_name')['date'].nunique()

# Find the product_name with the most date observations
most_observed_product = product_date_counts.sort_values(ascending=False).head(10)
most_observed_product

product_name
Bala de gelatina sticks ácido sabor morango                14
Bala de gelatina Sticks sabor framboesa                    14
Bala de gelatina sabor morango e framboesa Dentinhos       14
Bala gelatina ursinho de ouro                              14
Bala de gelatina sabor frutas sortidas Ursinhos de Ouro    14
Bala de gelatina sabor banana                              14
Bala sticks sabor morango ácido                            14
Bala de gelatina gotinhas                                  14
Bala de gelatina Wummies zourr sabor frutas                14
Bala de gelatina Ursinhos de Ouro sabor frutas             14
Name: date, dtype: int64

### Top 10 Most Frequent Product Names

In [66]:
# Find the most frequent product_name
top10_frequent_product = df_gold['product_name'].value_counts().head(10)
top10_frequent_product

product_name
Fio dental essencial menta                  942
Enxaguante bucal Cool Mint                  861
Creme dental tripla ação menta original     686
Salgadinho de milho sabor queijo nacho      685
Salgadinho de milho sabor presunto          667
Bebida láctea UHT sabor coco batata-doce    631
Refrigerante sabor original                 624
Sabonete líquido glicerinado Bebê           621
Cereal infantil banana, maçã e quinoa       559
Creme dental triple hortelã                 525
Name: count, dtype: int64

### Select one product

In [139]:
# select the brand
df_haribo = df_gold.query('brand == "Haribo"')
df_haribo

,date,aisle_name,product_name,brand,package,search_term,price,store_name,store_city
340,2023-07-26,Balas de gelatina,Bala de gelatina sabor morango e framboesa Den...,Haribo,Pacote 90g,Haribo,4.99,Atacadão,FLN
341,2023-07-26,Balas de gelatina,Bala de gelatina Ursinhos de Ouro sabor frutas,Haribo,Pacote 90g,Haribo,4.99,Atacadão,FLN
342,2023-07-26,Balas de gelatina,Bala de gelatina sabor frutas sortidas Ursinho...,Haribo,Pacote 60g,Haribo,2.99,Atacadão,FLN
343,2023-07-26,Balas de gelatina,Bala de gelatina sabor morango e nata Cream Kiss,Haribo,Pacote 90g,Haribo,4.99,Atacadão,FLN
344,2023-07-26,Balas de gelatina,Bala sticks sabor morango ácido,Haribo,80g,Haribo,4.99,Atacadão,FLN
...,...,...,...,...,...,...,...,...,...
244023,2023-08-13,Balas de gelatina,Bala de gelatina Sticks ácido sabor tutti,Haribo,80g,Haribo,7.29,Carrefour Hiper,FLN
244024,2023-08-13,Balas de gelatina,Marshmallow Chamallows sabor morango,Haribo,80g,Haribo,7.29,Carrefour Hiper,FLN
244025,2023-08-13,Balas de gelatina,Bala de gelatina Sticks sabor framboesa,Haribo,80g,Haribo,7.29,Carrefour Hiper,FLN
244026,2023-08-13,Balas de gelatina,Bala de gelatina sabor banana,Haribo,80g,Haribo,5.99,Carrefour Hiper,FLN


#### Clean `package` column

In [140]:
df_haribo.package.unique()

array(['Pacote 90g', 'Pacote 60g', '80g', '100g', 'Pacote 80g', '250g',
       '50g'], dtype=object)

In [141]:
df_haribo.loc[:, 'package'] = df_haribo['package'].str.replace('Pacote|g', '', regex=True).astype('int')

#### Filter for Acid Sticks

In [143]:
# filter the DataFrame and return rows where the 'product_name' column contains the string 'stick'
df_haribo_sticks = df_haribo[df_haribo['product_name'].str.contains('stick', case=False) & df_haribo['product_name'].str.contains('ácido', case=False)]
df_haribo_sticks.head(5)

,date,aisle_name,product_name,brand,package,search_term,price,store_name,store_city
344,2023-07-26,Balas de gelatina,Bala sticks sabor morango ácido,Haribo,80,Haribo,4.99,Atacadão,FLN
349,2023-07-26,Balas de gelatina,Bala de gelatina sticks ácido sabor morango,Haribo,80,Haribo,6.29,Angeloni Supermercados,FLN
356,2023-07-26,Balas de gelatina,Bala de gelatina Sticks ácido sabor tutti,Haribo,80,Haribo,7.29,BIG by Carrefour,FLN
358,2023-07-26,Balas de gelatina,Bala de gelatina Sticks ácido sabor framboesa,Haribo,80,Haribo,7.29,BIG by Carrefour,FLN
363,2023-07-26,Balas de gelatina,Bala de gelatina Sticks ácido sabor tutti,Haribo,80,Haribo,7.29,Carrefour Hiper,FLN


#### Turn dataframe to TS data

In [148]:
grouped = df_haribo_sticks.groupby(['store_name', 'date'])['price'].mean().reset_index()
grouped

,store_name,date,price
0,Angeloni Supermercados,2023-07-26,6.29
1,Angeloni Supermercados,2023-07-27,6.29
2,Angeloni Supermercados,2023-07-28,6.29
3,Angeloni Supermercados,2023-07-30,6.29
4,Angeloni Supermercados,2023-07-31,6.29
5,Angeloni Supermercados,2023-08-02,6.29
6,Angeloni Supermercados,2023-08-03,6.29
7,Angeloni Supermercados,2023-08-04,6.29
8,Angeloni Supermercados,2023-08-05,6.29
9,Angeloni Supermercados,2023-08-06,6.29


In [149]:
ts_grouped = grouped.pivot(index='date', columns='store_name', values='price')
ts_grouped.head()

store_name,Angeloni Supermercados,Atacadão,BIG by Carrefour,Carrefour Hiper
date,,,,
2023-07-26,6.29,4.99,7.29,7.29
2023-07-27,6.29,4.99,7.29,7.29
2023-07-28,6.29,4.99,7.29,7.29
2023-07-30,6.29,4.99,7.29,7.29
2023-07-31,6.29,4.99,7.29,7.29


### Price over time

In [150]:
import plotly.express as px

fig = px.line(ts_grouped, title='Time Series Plot of Prices Across Stores',
              labels={'date': 'Date', 'price': 'Price'},)

# Show the plot
fig.show()